# Eddy current analysis and reduced matrix

This example shows how to leverage PyAEDT to assign a matrix
and perform series or parallel connections in a Maxwell 2D design.

Keywords: **Maxwell 2D**, **eddy current**, *matrix**.

## Perform imports and define constants

Perform required imports.

In [1]:
import tempfile
import time

import ansys.aedt.core
from ansys.aedt.core.examples.downloads import download_file

Define constants.

In [2]:
AEDT_VERSION = "2025.2"
NUM_CORES = 4
NG_MODE = False  # Open AEDT UI when it is launched.

## Create temporary directory

Create a temporary directory where downloaded data or
dumped data can be stored.
If you'd like to retrieve the project data for subsequent use,
the temporary folder name is given by ``temp_folder.name``.

In [3]:
temp_folder = tempfile.TemporaryDirectory(suffix=".ansys")

## Download AEDT file

Set the local temporary folder to export the AEDT file to.

In [4]:
project_path = download_file(
    source="maxwell_ec_reduced_matrix",
    name="m2d_eddy_current.aedt",
    local_path=temp_folder.name,
)

## Launch AEDT and Maxwell 2D

Launch AEDT and Maxwell 2D, providing the version, path to the project, and the graphical mode.

In [5]:
m2d = ansys.aedt.core.Maxwell2d(
    project=project_path,
    version=AEDT_VERSION,
    design="EC_Planar",
    non_graphical=NG_MODE,
)

PyAEDT INFO: Parsing C:\Users\ansys\AppData\Local\Temp\tmp4mu_9yaz.ansys\maxwell_ec_reduced_matrix\m2d_eddy_current.aedt.


PyAEDT INFO: Python version 3.10.11 (tags/v3.10.11:7d4cc5a, Apr  5 2023, 00:38:17) [MSC v.1929 64 bit (AMD64)].


PyAEDT INFO: PyAEDT version 0.25.dev0.


PyAEDT INFO: Initializing new Desktop session.


PyAEDT INFO: Log on console is enabled.


...

PyAEDT WARNING: Please download and install latest Service Pack to use connect to AEDT in Secure Mode.


PyAEDT INFO: Debug logger is disabled. PyAEDT methods will not be logged.


PyAEDT INFO: Project m2d_eddy_current has been opened.


PyAEDT INFO: Active Design set to EC_Planar


PyAEDT INFO: Aedt Objects correctly read


## Assign a matrix

Assign a matrix given the list of sources to assign the matrix to and the return path.

In [6]:
matrix = m2d.assign_matrix(
    assignment=["pri", "sec", "terz"], matrix_name="Matrix1", return_path="infinite"
)

PyAEDT INFO: Modeler2D class has been initialized!


PyAEDT INFO: Modeler class has been initialized! Elapsed time: 0m 1sec


PyAEDT INFO: Infinite is the only return path option in EddyCurrent.


PyAEDT INFO: Boundary Matrix Matrix1 has been created.


## Assign reduced matrices

Assign reduced matrices to the parent matrix previously created.
For 2D/3D Eddy current solvers, two or more excitations can be joined
either in series or parallel connection. The result is known as a reduced matrix.

In [7]:
series = matrix.join_series(sources=["pri", "sec"], matrix_name="ReducedMatrix1")
parallel = matrix.join_parallel(sources=["sec", "terz"], matrix_name="ReducedMatrix2")

## Analyze setup

Run the analysis.

In [8]:
m2d.save_project()
m2d.analyze(setup=m2d.setup_names[0], cores=NUM_CORES, use_auto_settings=False)

PyAEDT INFO: Project m2d_eddy_current Saved correctly


PyAEDT INFO: Project m2d_eddy_current Saved correctly


PyAEDT INFO: Key Desktop/ActiveDSOConfigurations/Maxwell 2D correctly changed.


PyAEDT INFO: Solving design setup Setup1


PyAEDT INFO: Design setup Setup1 solved correctly in 0.0h 0.0m 26.0s


PyAEDT INFO: Key Desktop/ActiveDSOConfigurations/Maxwell 2D correctly changed.


True

## Get expressions

Get the available report quantities given the context
and the quantities category ``L``.

In [9]:
expressions = m2d.post.available_report_quantities(
    report_category="EddyCurrent",
    display_type="Data Table",
    context={"Matrix1": "ReducedMatrix1"},
    quantities_category="L",
)

PyAEDT INFO: Parsing C:\Users\ansys\AppData\Local\Temp\tmp4mu_9yaz.ansys\maxwell_ec_reduced_matrix\m2d_eddy_current.aedt.


PyAEDT INFO: File C:\Users\ansys\AppData\Local\Temp\tmp4mu_9yaz.ansys\maxwell_ec_reduced_matrix\m2d_eddy_current.aedt correctly loaded. Elapsed time: 0m 0sec


PyAEDT INFO: aedt file load time 0.04736900329589844


PyAEDT INFO: PostProcessor class has been initialized! Elapsed time: 0m 0sec


PyAEDT INFO: PostProcessor class has been initialized! Elapsed time: 0m 0sec


PyAEDT INFO: Post class has been initialized! Elapsed time: 0m 0sec


PyAEDT WARNING: Change the report category to AC Magnetic.


## Create report and get solution data

Create a data table report and get report data given the matrix context.

In [10]:
report = m2d.post.create_report(
    expressions=expressions,
    context={"Matrix1": "ReducedMatrix1"},
    plot_type="Data Table",
    setup_sweep_name="Setup1 : LastAdaptive",
    plot_name="reduced_matrix",
)
data = m2d.post.get_solution_data(
    expressions=expressions,
    context={"Matrix1": "ReducedMatrix1"},
    report_category="EddyCurrent",
    setup_sweep_name="Setup1 : LastAdaptive",
)

PyAEDT WARNING: No report category provided. Automatically identified AC Magnetic


PyAEDT INFO: Solution Data Correctly Loaded.


Time to initialize solution data:0.0073812007904052734
Time to initialize solution data:0.023161649703979492


## Get matrix data

Get inductance results for the join connections in ``nH``.

In [11]:
ind = ansys.aedt.core.generic.constants.unit_converter(
    data.get_expression_data(formula="magnitude")[1][0],
    unit_system="Inductance",
    input_units=data.units_data[expressions[0]],
    output_units="uH",
)

## Release AEDT

In [12]:
m2d.save_project()
m2d.release_desktop()
# Wait 3 seconds to allow AEDT to shut down before cleaning the temporary directory.
time.sleep(3)

PyAEDT INFO: Project m2d_eddy_current Saved correctly


PyAEDT INFO: Desktop has been released and closed.


## Clean up

All project files are saved in the folder ``temp_folder.name``.
If you've run this example as a Jupyter notebook, you
can retrieve those project files. The following cell
removes all temporary files, including the project folder.

In [13]:
temp_folder.cleanup()